## Fraud Detection using Machine Learning
##### using a logistic regression model to predict binary outcome of wheteher fruad is detected in a statement

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
data = pd.read_csv('fraudTrain.csv')

get rid of uselss data

In [4]:
data.drop(columns = ['Unnamed: 0','trans_date_trans_time','cc_num','merchant','first','last','street','city','zip','lat','long','dob','trans_num','unix_time', 'merch_lat', 'merch_long'], inplace = True)

In [5]:
data.head(5)

,category,amt,gender,state,city_pop,job,is_fraud
0,misc_net,4.97,F,NC,3495,"Psychologist, counselling",0
1,grocery_pos,107.23,F,WA,149,Special educational needs teacher,0
2,entertainment,220.11,M,ID,4154,Nature conservation officer,0
3,gas_transport,45.00,M,MT,1939,Patent attorney,0
4,misc_pos,41.96,M,VA,99,Dance movement psychotherapist,0


convert strings to integers 

In [6]:
categoryData = (pd.get_dummies(data['category']).astype(int))
categoryData.head()

,entertainment,food_dining,gas_transport,grocery_net,grocery_pos,health_fitness,home,kids_pets,misc_net,misc_pos,personal_care,shopping_net,shopping_pos,travel
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [7]:
data.drop(columns=['category'], inplace=True)

In [8]:
data = pd.concat([data, categoryData], axis=1)

In [9]:
data['gender'].replace({'F': 0, 'M': 1}, inplace=True)

In [10]:
stateData = (pd.get_dummies(data['state']).astype(int))
data.drop(columns=['state'], inplace=True)
data = pd.concat([data, stateData], axis=1)

In [11]:
jobData = (pd.get_dummies(data['job']).astype(int))
data.drop(columns=['job'], inplace=True)
data = pd.concat([data, jobData], axis=1)

// Divide the data into features and labels

In [12]:

X = data.drop(columns=['is_fraud'])
y = data['is_fraud']

In [13]:
y.value_counts()

is_fraud
0    1289169
1       7506
Name: count, dtype: int64

Handle imbalance dataset - smote https://github.com/SoumyadeepB/Machine-Learning/blob/master/Model%20Improvement%20Techniques/SMOTE%20for%20Imbalanced%20Datasets.ipynb

In [14]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='minority')

In [15]:
X , y = smote.fit_resample(X, y)

In [16]:
y.value_counts()

is_fraud
0    1289169
1    1289169
Name: count, dtype: int64

normalize with max value of 1

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
X[:] = scaler.fit_transform(X)


train test split

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)


In [22]:
X_train.shape, X_test.shape

((2062670, 562), (515668, 562))

model 

In [26]:
model = tf.keras.models.Sequential([
    #input layer (dense means it needs to be fully connected)
    tf.keras.layers.Dense(300, activation='relu', input_shape=(562,)),
    #hideen layer 
    tf.keras.layers.Dense(200, activation='relu'),
    #output layer (1 means it is binary classification)
    tf.keras.layers.Dense(1, activation='sigmoid')     
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=200, validation_data=(X_test, y_test))

c:\Users\luthi\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
10314/10314 ━━━━━━━━━━━━━━━━━━━━ 44s 4ms/step - accuracy: 0.9623 - loss: 0.0907 - val_accuracy: 0.9880 - val_loss: 0.0309
Epoch 2/10
10314/10314 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - accuracy: 0.9884 - loss: 0.0281 - val_accuracy: 0.9897 - val_loss: 0.0254
Epoch 3/10
10314/10314 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.9903 - loss: 0.0231 - val_accuracy: 0.9908 - val_loss: 0.0229
Epoch 4/10
10314/10314 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.9913 - loss: 0.0207 - val_accuracy: 0.9920 - val_loss: 0.0201
Epoch 5/10
10314/10314 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.9921 - loss: 0.0187 - val_accuracy: 0.9928 - val_loss: 0.0196
Epoch 6/10
10314/10314 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - accuracy: 0.9929 - loss: 0.0174 - val_accuracy: 0.9930 - val_loss: 0.0179
Epoch 7/10
10314/10314 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - accuracy: 0.9934 - loss: 0.0162 - val_accuracy: 0.9932 - val_loss: 0.0179
Epoch 8/10
10314/10314 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - accuracy: 

In [27]:
loss,accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

16115/16115 ━━━━━━━━━━━━━━━━━━━━ 14s 847us/step - accuracy: 0.9938 - loss: 0.0160
Accuracy: 99.40


In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 300)            │       168,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │        60,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 687,905 (2.62 MB)

 Trainable params: 229,301 (895.71 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 458,604 (1.75 MB)

make predictions - from website idk what this person was doing im ngl

In [30]:
predicted = model.predict(X_test)


16115/16115 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step


In [33]:
binary_predicted = ((predicted > 0.5)).astype('int')
binary_predicted[:5]
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test, binary_predicted))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99    257834
           1       0.99      0.99      0.99    257834

    accuracy                           0.99    515668
   macro avg       0.99      0.99      0.99    515668
weighted avg       0.99      0.99      0.99    515668



In [34]:
#save model 
model.save('model.h5')